In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.feature_selection import SelectFpr, f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from scipy.stats import chi2_contingency
from datetime import date


In [3]:
df = pd.read_csv('credit_risk_dataset (1).csv')
df.head()

C:\Users\Hp\AppData\Local\Temp\ipykernel_14504\3973402256.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('credit_risk_dataset (1).csv')


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
nullCol = df.isnull().sum()
nullCol

id                       0
member_id                0
loan_amnt                0
funded_amnt              0
funded_amnt_inv          0
                     ...  
all_util            466285
total_rev_hi_lim     70276
inq_fi              466285
total_cu_tl         466285
inq_last_12m        466285
Length: 74, dtype: int64

In [5]:
df = df.dropna(axis=1,thresh = 320000)
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,...,Jan-15,171.62,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,...,Apr-13,119.66,Sep-13,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,...,Jun-14,649.91,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,...,Jan-15,357.48,Jan-15,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,...,Jan-16,67.79,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN


In [6]:
def getDuplicateColumns(df):
    
    
    duplicateColumnNames = set()
    
    for x in range(df.shape[1]):
    
        col = df.iloc[:, x]

        for y in range(x + 1, df.shape[1]):
            
            otherCol = df.iloc[:, y]
            
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
    return list(duplicateColumnNames)
df = df.drop(columns=getDuplicateColumns(df))
df


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,...,Jan-15,171.62,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,...,Apr-13,119.66,Sep-13,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,...,Jun-14,649.91,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,...,Jan-15,357.48,Jan-15,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,...,Jan-16,67.79,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466280,8598660,1440975,18400,18400,18400.0,60 months,14.47,432.64,C,C2,...,Jan-16,432.64,Jan-16,0.0,1,INDIVIDUAL,0.0,0.0,294998.0,29900.0
466281,9684700,11536848,22000,22000,22000.0,60 months,19.97,582.50,D,D5,...,Dec-14,17.50,Jan-16,0.0,1,INDIVIDUAL,0.0,0.0,221830.0,39400.0
466282,9584776,11436914,20700,20700,20700.0,60 months,16.99,514.34,D,D1,...,Jan-16,514.34,Dec-15,0.0,1,INDIVIDUAL,0.0,0.0,73598.0,13100.0
466283,9604874,11457002,2000,2000,2000.0,36 months,7.90,62.59,A,A4,...,Dec-14,1500.68,Apr-15,0.0,1,INDIVIDUAL,0.0,0.0,591610.0,53100.0


In [7]:
my_list = list(df)
print(my_list)

['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim']


In [8]:
#I have chosen Loan_Status as my target variable because by use of other attributes we can create a model and then we can test on this target column.


In [9]:
df['Target'] = (df['loan_status'] != "Charged Off")
df['Target']

0          True
1         False
2          True
3          True
4          True
          ...  
466280     True
466281    False
466282     True
466283     True
466284     True
Name: Target, Length: 466285, dtype: bool

In [10]:
df=df.drop(['policy_code','initial_list_status','pub_rec','int_rate','installment','collection_recovery_fee','total_rec_prncp','total_rec_late_fee','emp_title','url','sub_grade','id','member_id'],axis=1)


In [11]:
y=df.Target
x=df.drop('Target',axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
print(x_train.shape)
print(x_test.shape)

(373028, 39)
(93257, 39)


In [12]:
df['issue_date'] = [0 if pd.isna(x) else x for x in df['issue_d']]
df['earliest_cr_line'] = [0 if pd.isna(x) else x for x in df['earliest_cr_line']]
df['last_pymnt_d'] = [0 if pd.isna(x) else x for x in df['last_pymnt_d']]
df['last_credit_pull_d'] = [0 if pd.isna(x) else x for x in df['last_credit_pull_d']]
#Converted NaN date values to 0

In [13]:
df['issue_date']

0         Dec-11
1         Dec-11
2         Dec-11
3         Dec-11
4         Dec-11
           ...  
466280    Jan-14
466281    Jan-14
466282    Jan-14
466283    Jan-14
466284    Jan-14
Name: issue_date, Length: 466285, dtype: object

In [14]:
def date_conv(date):
    s = 0
    if(date[:3]=="Jan"):
        s=11
    elif(date[:3]=="Feb"):
        s=10
    elif(date[:3]=="Mar"):
        s=9
    elif(date[:3]=="Apr"):
        s=8
    elif(date[:3]=="May"):
        s=7
    elif(date[:3]=="Jun"):
        s=6
    elif(date[:3]=="Jul"):
        s=5
    elif(date[:3]=="Aug"):
        s=4
    elif(date[:3]=="Sep"):
        s=3
    elif(date[:3]=="Oct"):
        s=2
    elif(date[:3]=="Nov"):
        s=1
    elif(date[:3]=="Dec"):
        s=0
    s =s + (20 -int(date[4:8]))*12
    return s
x_train['issue_d_diff'] = [0 if x==0 else date_conv(x) for x in x_train['issue_d']]
x_train['issue_d_diff']
#converted date to numeric form(months)
        

375372     78
18168     117
265747     73
365113     78
4880      110
         ... 
147376     92
214140    102
5422      110
58282      85
234635     72
Name: issue_d_diff, Length: 373028, dtype: int64